# 目标
使用DeepSpeed调整ZeRO-3配置文件支持T5-3B模型训练

## 训练命令

In [ ]:
# DeepSpeed ZeRO-3 模式单 GPU 训练翻译模型（T5-Large）
deepspeed --num_gpus=1 translation/run_translation.py \
--deepspeed config/ds_config_zero3.json \
--model_name_or_path t5-3b --per_device_train_batch_size 1 \
--output_dir output_dir --overwrite_output_dir --fp16 \
--do_train --max_train_samples 500 --num_train_epochs 1 \
--dataset_name wmt16 --dataset_config "ro-en" \
--source_lang en --target_lang ro

## 参数配置调整
### 调整1
将stage3_max_live_parameters参数调大未1e10

### 调整2
去除offload_param参数配置,在offload_optimizer中增加ratio参数配置


### 调整后的参数如下

In [4]:
import json

# 打开 JSON 文件
with open('./config/ds_config_zero3.json', 'r') as file:
    # 读取文件内容并解析为字典
    data = json.load(file)

# 打印字典内容
print(json.dumps(data, indent=4))

{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "bf16": {
        "enabled": "auto"
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true,
            "ratio": 0.9
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1000000000.0,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": 

## 执行结果
在开始训练后，执行失败，上述参数调整后，在尽可能大的使用GPU显存的情况下，剩余部分使用CPU内存，CPU内存配置为32G+64G虚拟内存，
当CPU虚拟内存使用到11G左右时，由于硬盘的读写速度远小于内存，系统处理性能会显著下降，系统会变得非常缓慢直至无法响应，然后任务停止无法继续执行，执行截图如下
![](translation/deepspeedtrain.png)

目前还没有找到其他方法可以解决问题，希望老师进一步指导！